In [ ]:
# Import relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import urllib.request, json

## /ListSensors

In [ ]:
with open('ListSensor.txt', encoding="utf8") as f:
    contents = f.read()

In [ ]:
vecname = ["SiteCode","SiteName","Latitude","Longitude","LocalAuthorityID","SiteClassification",
           "HeadHeight","ToRoad","SiteLocationType","Indoor","SitePhotoURL","SiteDescription",
           "SleepTime","BatteryStatus","BatteryPercentage","SignalStrength","SensorsHealthStatus",
           "OverallStatus","DeviceCode","StartDate","EndDate","LastCommunication","InputPowerError", 
           "InputPowerCurrent", "InputPowerVoltage","InputBatteryError","InputBatteryVoltage",
           "ChargingStatus","SignalQuality","BatterySleepMultiplier","WeatherError",
           "PMError","NO2Error",
           "LatestINO2Value","LatestINO2Index","LatestINO2IndexSource",
           "LatestIPM10Value","LatestIPM10NUMValue","LatestIPM1Value","LatestIPM1NUMValue",
           "LatestIPM25Value","LatestIPM25Index","LatestIPM25IndexSource","LatestIPM25NUMValue",
           "LatestRELHUMValue","LatestTEMPERATValue","SiteActive","SiteGroup","PowerTag","Enabled","OtherTags",
           "OrganisationName","SponsorName","HourlyBulletinEnd"]
indstart = contents.find("SiteCode")
nsite = 346
vecstart = np.zeros([nsite])*np.nan
vecstart[0] = indstart
for i in np.arange(1,nsite):
    vecstart[i] = int(contents.find("SiteCode",int(vecstart[i-1]+8),-1))


In [ ]:
dfdata = pd.DataFrame(columns=vecname)
for i in np.arange(0,nsite):
    print(i)
    datatemp = np.array([])
    istart = int(vecstart[i]-1)
    if i == nsite-1:
        iend = len(contents)-3
    else:
        iend = int(vecstart[i+1]-4)
    strtemp = contents[istart:iend]
    for j in np.arange(0,len(vecname)-1):
        nametemp1 = vecname[j]
        nametemp2 = vecname[j+1]    
        lentemp1 = len(nametemp1)
        ivaluetemp1 = strtemp.find(nametemp1)+lentemp1+2
        if strtemp[ivaluetemp1] == '\"':
            ivaluetemp1 = strtemp.find(nametemp1)+lentemp1+3
            ivaluetemp2 = strtemp.find(nametemp2)-3
        else:
            ivaluetemp1 = strtemp.find(nametemp1)+lentemp1+2
            ivaluetemp2 = strtemp.find(nametemp2)-2
        valuetemp = strtemp[ivaluetemp1:ivaluetemp2]
        datatemp = np.append(datatemp,valuetemp)
    j = len(vecname)-1
    nametemp1 = vecname[j]
    lentemp1 = len(nametemp1)
    ivaluetemp1 = strtemp.find(nametemp1)+lentemp1+3
    ivaluetemp2 = len(strtemp)-1
    valuetemp = strtemp[ivaluetemp1:ivaluetemp2]
    datatemp = np.append(datatemp,valuetemp)
    
    
    dfdata.loc[i] = datatemp

In [ ]:
dfdata.to_csv('ListSensors.csv')

In [ ]:
dfdata = pd.read_csv("ListSensors.csv")

## /getClarityData

### update time parameter

In [ ]:
for i in np.arange(0,1):
    sitecode = dfdata['SiteCode'][i]
    startdate = dfdata['StartDate'][i],
    enddate = dfdata['LastCommunication'][i]

In [ ]:
totd = 410 #update number of days
toth = totd*24
dti = pd.date_range("2022-01-01", periods=toth, freq="H")

In [ ]:
startdate = ('2022-01-01T00')

In [ ]:
dfmattime = pd.DataFrame(columns=['Weekday', 'Day', 'Month','Year','Hour'])
for i in np.arange(0,toth):
    print(i)
    titemp = str(dti[i])
        
    if dti[i].isoweekday()==1:
        wdtemp = 'Mon'
    elif dti[i].isoweekday()==2:
        wdtemp = 'Tue'
    elif dti[i].isoweekday()==3:
        wdtemp = 'Wed'
    elif dti[i].isoweekday()==4:
        wdtemp = 'Thu'
    elif dti[i].isoweekday()==5:
        wdtemp = 'Fri'
    elif dti[i].isoweekday()==6:
        wdtemp = 'Sat'
    elif dti[i].isoweekday()==7:
        wdtemp = 'Sun'
     
    mtemp = int(titemp[5:7])
    if mtemp == 1:
        mstrtemp = 'Jan'
    elif mtemp == 2:
        mstrtemp = 'Feb'
    elif mtemp == 3:
        mstrtemp = 'Mar'
    elif mtemp == 4:
        mstrtemp = 'Apr'
    elif mtemp == 5:
        mstrtemp = 'May'
    elif mtemp == 6:
        mstrtemp = 'Jun'
    elif mtemp == 7:
        mstrtemp = 'Jul'
    elif mtemp == 8:
        mstrtemp = 'Aug'
    elif mtemp == 9:
        mstrtemp = 'Sep'
    elif mtemp == 10:
        mstrtemp = 'Oct'
    elif mtemp == 11:
        mstrtemp = 'Nov'
    elif mtemp == 12:
        mstrtemp = 'Dec'
        
    datatemp = np.array([wdtemp,int(titemp[8:10]),mstrtemp,int(titemp[0:4]),titemp[11:19]]) 
    
    dfmattime.loc[i] = datatemp


### get raw data

In [ ]:
matpmdata = pd.DataFrame(columns=list(dfdata['SiteCode']), index=dti)
for i in np.arange(29, len(dfdata)):
    sitecode = dfdata['SiteCode'][i]    
    #startdate = dfdata['StartDate'][i]
    enddate = dfdata['LastCommunication'][i]
    if enddate=="null" or pd.isnull(enddate):
        enddate = dfdata['EndDate'][i]

    y1 = startdate[0:4]
    m1 = str(int(startdate[5:7]))
    if m1 == "1":
        m1 = 'Jan'
    elif m1 == "2":
        m1 = 'Feb'
    elif m1 == "3":
        m1 = 'Mar'
    elif m1 == "4":
        m1 = 'Apr'
    elif m1 == "5":
        m1 = 'May'
    elif m1 == "6":
        m1 = 'Jun'
    elif m1 == "7":
        m1 = 'Jul'
    elif m1 == "8":
        m1 = 'Aug'
    elif m1 == "9":
        m1 = 'Sep'
    elif m1 == "10":
        m1 = 'Oct'
    elif m1 == "11":
        m1 = 'Nov'
    elif m1 == "12":
        m1 = 'Dec'
    d1 = str(int(startdate[8:10]))
    h1 = startdate[11:13]+':00:00'
    
    y2 = enddate[0:4]
    m2 = str(int(enddate[5:7]))
    if m2 == "1":
        m2 = 'Jan'
    elif m2 == "2":
        m2 = 'Feb'
    elif m2 == "3":
        m2 = 'Mar'
    elif m2 == "4":
        m2 = 'Apr'
    elif m2 == "5":
        m2 = 'May'
    elif m2 == "6":
        m2 = 'Jun'
    elif m2 == "7":
        m2 = 'Jul'
    elif m2 == "8":
        m2 = 'Aug'
    elif m2 == "9":
        m2 = 'Sep'
    elif m2 == "10":
        m2 = 'Oct'
    elif m2 == "11":
        m2 = 'Nov'
    elif m2 == "12":
        m2 = 'Dec'
    d2 = str(int(enddate[8:10]))
    h2 = enddate[11:13]+':00:00'
    
    if y2 >= y1:
        ind1 = dfmattime[(dfmattime['Year']==y1) & (dfmattime['Month']==m1) & (dfmattime['Day']==d1) & (dfmattime['Hour']==h1)].index[0]
        ind2 = dfmattime[(dfmattime['Year']==y2) & (dfmattime['Month']==m2) & (dfmattime['Day']==d2) & (dfmattime['Hour']==h2)].index[0]

        url = ("https://api.breathelondon.org/api/getClarityData/" + sitecode + "/INO2/" 
               + dfmattime['Weekday'][ind1] + "%20" + dfmattime['Month'][ind1] + "%20" 
               + dfmattime['Year'][ind1] + "%20" + dfmattime['Hour'][ind1] 
               + "%20GMT/" 
               + dfmattime['Weekday'][ind2] + "%20" + dfmattime['Month'][ind2] + "%20" 
               + dfmattime['Year'][ind2] + "%20" + dfmattime['Hour'][ind2]
               + "%20GMT/Hourly?key=fe476fea-e87a-11eb-9a03-0242ac130003") # add own api key

        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            if len(data)>4:

                for j in np.arange(0,len(data)):
                    print(f'{i}, {j}/{len(data)}')

                    datetemp = data[j]['DateTime']


                    y = datetemp[0:4]
                    m = str(int(datetemp[5:7]))
                    if m == "1":
                        m = 'Jan'
                    elif m == "2":
                        m = 'Feb'
                    elif m == "3":
                        m = 'Mar'
                    elif m == "4":
                        m = 'Apr'
                    elif m == "5":
                        m = 'May'
                    elif m == "6":
                        m = 'Jun'
                    elif m == "7":
                        m = 'Jul'
                    elif m == "8":
                        m = 'Aug'
                    elif m == "9":
                        m = 'Sep'
                    elif m == "10":
                        m = 'Oct'
                    elif m == "11":
                        m = 'Nov'
                    elif m == "12":
                        m = 'Dec'
                    d = str(int(datetemp[8:10]))
                    h = datetemp[11:13]+':00:00'
                    indr = dfmattime[(dfmattime['Year']==y) & (dfmattime['Month']==m) & (dfmattime['Day']==d) & (dfmattime['Hour']==h)].index[0]
                    indc = list(dfdata['SiteCode']).index(sitecode)
                    matpmdata.iloc[indr,indc] = data[j]['ScaledValue']


In [ ]:
matpmdata.to_csv('no2data_full.csv')